<a href="https://colab.research.google.com/github/Viktory4121/BKS_py/blob/main/lab2_bks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [295]:
import random

In [296]:
#входные данные

#входное сообщение:
message = "Import message."

#ключ из 8 четырёхзначных чисел
key = [1123, 350, 6431, 9202, 7265, 3304, 7782, 9955]

In [297]:
def generate_matrix():
  table = [] #таблица на 33 строки и 16 столбцов

  for i in range(33):
    nums = list(range(16)) #создание списка чисел от 0 до 15
    random.shuffle(nums) #производится случайная расстановка вышесозданных чисел
    new_row = []
    for k in nums: #добавление случайно перестановленных чисел в таблицу
      new_row.append(k)
    
    table.append(new_row.copy())
    new_row.clear()
  
  return table

In [298]:
def add(a, b): #сложение по модулю 2 в 32 степени
  return bin((int(a, base=2) + b) % pow(2, 32)).zfill(32)

In [299]:
def f(num, table):
  binary = num[2:].zfill(32)
  arr_bin = [binary[i:i + 4] for i in range(0, len(binary), 4)] #двоичная строка из 32 символов делится по 4 символа и формирует массив
  new_num = ""
  
  for i in range(len(arr_bin)):
    for k in range(len(table)):
      if k == int(arr_bin[i], base=2):
        new_num += (bin(table[k])[2:].zfill(4))
        break
  
  return new_num.zfill(32)

In [300]:
def Coding(left, right, key, table):
  T_L = []
  T_R = []

  T_new_L = ""
  T_new_R = ""

  arr_T_L = []
  arr_T_R = []

  rezult_str = []

  T_L.append(left)
  T_R.append(right)
  for i in range(1, 33): #цикл зашифрования
    if i <= 24:
      T_L.append(bin(int(f(add(T_L[i-1], key[(i-1)%8]), table[i]), base=2) ^ int(T_R[i-1], base=2))[2:].ljust(32, '0'))
      T_R.append(T_L[i-1])
    elif i > 24 and i <= 31:
      T_L.append(bin(int(f(add(T_L[i-1], key[31-i]), table[i]), base=2) ^ int(T_R[i-1], base=2))[2:].ljust(32, '0'))
      T_R.append(T_L[i-1])
    elif i == 32:
      T_new_L = T_L[i-1]
      T_new_R = bin(int(f(add(T_L[i-1], key[0]), table[i]), base=2) ^ int(T_R[i-1], base=2))[2:].ljust(32, '0')

  
  #разбиение левой и правой части блока на массивы из чисел по 8 символов в двоичной системе:
  dop_left = [T_new_L[i:i + 8] for i in range(0, len(T_new_L), 8)]
  dop_right = [T_new_R[i:i + 8] for i in range(0, len(T_new_R), 8)]
  for k in range(len(dop_left)):
    #сбор строки и перевод её в символы через ascii:
    rezult_str.append(chr(int(dop_left[k], base=2)))
    rezult_str.append(chr(int(dop_right[k], base=2)))
    
  return rezult_str #массив символов, переведённых через ascii

In [301]:
def Block_cipher(message, key):
  matrix = generate_matrix() #генерация таблицы замен

  T_L = []
  T_R = []
  T_new = []
  count_blok = 0 #общее количество блоков, на которое делится сообщение
  mess = []
  bin_mess = []
  str_bin_mess = ""

  #разделение строки символов на массив
  if type(message) == str:
    for i in message:
      mess.append(i)
  else:
    mess = message.copy()
    
  #перевод символов в двоичную систему ascii
  k = 0
  for i in mess:
    bin_mess.append(bin(ord(i))) #перевод каждого символа в двоичный ascii-код
    #str.zfill(num) - дополнение нулями слева
    str_bin_mess += (bin_mess[k][2:]).zfill(8) #составление одного большого сообщения из двоичных чисел
    k += 1

  z = 0
  while(z < len(str_bin_mess)):
    #str.ljust(num, '0') - дополнение нулями справа
    T_L.append(str_bin_mess[z:(z+32)]) #32 бита
    T_R.append((str_bin_mess[(z+32):(z+64)]).ljust(32, '0')) #32 бита
    count_blok += 1 #1 блок содержит 64 бита
    z += 64

  for i in range(count_blok): #цикл по сформированным блокам
    dop_arr = Coding(T_L[i], T_R[i], key, matrix) #зашифровка каждого блока
    for i in dop_arr:
      T_new.append(i) #сборка зашифрованного сообщения в один массив

  return T_new

In [302]:
print("Зашифрованное сообщение:\n", Block_cipher(message, key))

Зашифрованное сообщение:
 ['è', '\x88', 'ý', 'u', 'Í', '8', '*', '1', 'Æ', 'É', '\x16', '½', '\x7f', 'Ò', 'ë', '\x8d']
